In [ ]:
# limit the thread used by numpy 
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"  
os.environ["MKL_NUM_THREADS"] = "1"

import numpy as np
import scipy.interpolate as interp 
try:
    import cupy as xp
    import cupyx.scipy.interpolate as xinterp
    # print("has cupy")
except (ImportError, ModuleNotFoundError) as e:
    import numpy as xp
    import scipy.interpolate as xinterp  
    # print("no cupy")

from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
# import matplotlib
# matplotlib.rcParams['text.usetex'] = True
# matplotlib.rcParams['font.family'] = 'serif'

from Triangle.Constants import *
from Triangle.Orbit import * 
from Triangle.Noise import *
from Triangle.FFTTools import *
from Triangle.TDI import XYZfromAET, AETfromXYZ

from Triangle_BBH.Waveform import * 
from Triangle_BBH.Response import *
from Triangle_BBH.Utils import *
from Triangle_BBH.Fisher import *

np.random.seed(114514)
xp.random.seed(114514)

## Load data 

In [ ]:
with open("FDSimulationData.pkl", "rb") as datafile:
    datadict = pickle.load(datafile)
datadict.keys()

In [ ]:
data_frequency = xp.array(datadict["frequency"])
delta_f = data_frequency[1] - data_frequency[0]
Tobs = 1. / delta_f

data_channels = xp.array(datadict["AET"])

fiducial_parameters = datadict["parameters"]

response_kwargs = datadict["response_kwargs"]

data_frequency.shape, data_channels.shape, response_kwargs

## Prepare models 

### orbit

In [ ]:
# orbit model 
orbit_file = '/home/TFDAnalysls_new/Data/OrbitData/MicroSateOrbitEclipticTCB'
orbit = Orbit(OrbitDir=orbit_file)

### waveform and response 

In [ ]:
mode = "primary"
use_gpu = True 

# initialize  waveform generator 
WFG = BBHxWaveformGenerator(mode=mode, use_gpu=use_gpu)

# initialize response generator 
FDTDI = BBHxFDTDIResponseGenerator(orbit_class=orbit, waveform_generator=WFG, use_gpu=use_gpu)

channel_names = ["A2", "E2", "T2"]

## Model noise 
notice that this is a simplified model and the AET channels are only approximately independent. 

In [ ]:
# noise PSD model 
PSDfunc = TDIPSDs()

# use the median time to calculate armlengths
arm_time = (response_kwargs["tmin"] + response_kwargs["tmax"]) / 2. * DAY 
arms = dict()
for key in MOSA_labels:
    arms[key] = orbit.LTTfunctions()[key](arm_time)
print("arm time (day):", arm_time / DAY)
print("arm lengths:", arms)

PSD_channels = xp.array([
    PSDfunc.PSD_A2_unequal(data_frequency.get(), arms), 
    PSDfunc.PSD_E2_unequal(data_frequency.get(), arms), 
    PSDfunc.PSD_T2_unequal(data_frequency.get(), arms)
])

# covariance matrix 
CovMat = xp.array([
    [PSD_channels[0], xp.zeros_like(PSD_channels[0]), xp.zeros_like(PSD_channels[0])],
    [xp.zeros_like(PSD_channels[0]), PSD_channels[1], xp.zeros_like(PSD_channels[0])],
    [xp.zeros_like(PSD_channels[0]), xp.zeros_like(PSD_channels[0]), PSD_channels[2]]
]) / 4. / delta_f # (3, 3, Nf)

# inverse of covmatrix
InvCovMat = xp.linalg.inv(xp.transpose(CovMat, (2, 0, 1))) # (Nf, 3, 3)

PSD_channels.shape, InvCovMat.shape 

In [ ]:
for ich, nch in enumerate(channel_names):
    plt.loglog(data_frequency.get(), np.abs(data_channels[ich].get()), label=nch, linewidth=1)
    plt.loglog(data_frequency.get(), np.sqrt(PSD_channels[ich] / 2. / delta_f).get(), color="grey")

plt.legend(loc="upper left")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Fourier transform (1/Hz)")
plt.grid(linestyle=":")

## Use A, E channels only 
set $C^{-1}_{T_2 T_2} \equiv 0$ to eliminate the contribution of $T_2$

In [ ]:
InvCovMat[:, 2, 2] *= 0. 

## Likelihood

In [ ]:
Like = Likelihood(
    response_generator=FDTDI, 
    frequency=data_frequency, 
    data=data_channels, 
    invserse_covariance_matrix=InvCovMat, 
    response_parameters=response_kwargs, 
    use_gpu=use_gpu,
)

### test if likelihood peaks at truths 

In [ ]:
fiducial_param_arr = ParamDict2ParamArr(fiducial_parameters)

N_test=101
test_idx = 4 # tc 
test_params = np.linspace(-1e-3, 1e-3, N_test) + fiducial_param_arr[test_idx]

tmp_params = np.zeros((11, N_test))
for i_test in range(N_test):
    tmp_params[:, i_test] = fiducial_param_arr.copy()
    tmp_params[test_idx][i_test] = test_params[i_test]
test_lls_vec = Like.full_log_like_vectorized(tmp_params)
plt.plot(test_params, test_lls_vec)

## Run MCMC Sampling 

the aim is to compare the posteriors of full and heterodyned likelihoods, so we choose ideal priors and starting points. 

In [ ]:
from eryn.ensemble import EnsembleSampler
from eryn.state import State
from eryn.prior import ProbDistContainer, uniform_dist
from eryn.utils import TransformContainer
from eryn.moves import GaussianMove, StretchMove, CombineMove
from eryn.utils.utility import groups_from_inds
from eryn.backends import HDFBackend
from eryn.utils import SearchConvergeStopping

from corner import corner

In [ ]:
labels = [r"${\rm lg}\mathcal{M}_{c,z}$", r"$q$", r"$\chi_{z,1}$", r"$\chi_{z,2}$", r"$t_c$", r"$\varphi_c$", r"${\rm lg} D_L$", r"$\cos \iota$", r"$\lambda$", r"$\sin \beta$", r"$\psi$"]

truths = ParamDict2ParamArr(fiducial_parameters)

truths

In [ ]:
# set hyper parameters
ndim = 11 # dimension of paramters 
nwalkers = 100 # number of random walkers, limited by the vRAM of my 4080S, use fewer (e.g. 100) to speed up and more (e.g. 400) to get more smooth posterior 
ntemps = 10 # number of temperatures used in parallel tempering 
temps = np.array(list(np.power(2., np.arange(ntemps - 1))) + [np.infty]) 
betas = 1. / temps 
tempering_kwargs=dict(betas=betas)

mcmc_moves = StretchMove(a=2) # mcmc move 

stop = None 

# set priors 
lim_lgMc = [5.5, 6.5]
lim_q = [0.1, 0.999]
lim_chiz1 = [-0.99, 0.99]
lim_chiz2 = [-0.99, 0.99]
lim_tc = [fiducial_parameters["coalescence_time"] - 500/DAY, fiducial_parameters["coalescence_time"] + 500/DAY] # assume a preliminary search step to locate the merger within 1000s
lim_phic = [0, TWOPI]
lim_lgD = [3.5, 5.5]
lim_cosinc = [-1, 1]
lim_lam = [0, TWOPI]
lim_sinbeta = [-1, 1]
lim_psi = [0, PI]

lims = np.array([lim_lgMc, lim_q, lim_chiz1, lim_chiz2, lim_tc, lim_phic, lim_lgD, lim_cosinc, lim_lam, lim_sinbeta, lim_psi])
lower_lims = lims[:, 0]
upper_lims = lims[:, 1]

priors_in = {i: uniform_dist(lims[i][0], lims[i][1]) for i in range(ndim)}
priors = ProbDistContainer(priors_in)
priors.use_cupy = False

# set starting range 
start_lims = np.array(truths)[:, np.newaxis] + np.array([-1e-3, 1e-3])
start_priors_in = {i: uniform_dist(start_lims[i][0], start_lims[i][1]) for i in range(ndim)}
start_priors = ProbDistContainer(start_priors_in)
start_priors.use_cupy = False

lims, start_lims

In [ ]:
def eryn_like(params): 
    """params: numpy array of shape (Nevents, Nparams)"""
    return Like.full_log_like_vectorized(np.transpose(params))

In [ ]:
# init_ensemble = EnsembleSampler(
#     nwalkers,
#     ndim,
#     eryn_like, 
#     priors,
#     args=[],
#     tempering_kwargs=tempering_kwargs,
#     stopping_fn=stop,
#     stopping_iterations=10,
#     moves=mcmc_moves,
#     vectorize=True,
# )

# filename='MCMCResults/FullLogLike.h5'
# backend = HDFBackend(filename)
# backend.reset(nwalkers=nwalkers, ndims=ndim, ntemps=ntemps, moves=init_ensemble.backend.move_keys)
# print('args of the backend =', backend.reset_args)
# print('move keys =', backend.move_keys)
# print('Initialization flag of backend =', backend.initialized)


# ensemble = EnsembleSampler(
#     nwalkers,
#     ndim,
#     eryn_like, 
#     priors,
#     args=[],
#     tempering_kwargs=tempering_kwargs,
#     stopping_fn=stop,
#     stopping_iterations=10,
#     moves=init_ensemble.moves,
#     backend=backend,
#     vectorize=True,
# )


ensemble = EnsembleSampler(
    nwalkers,
    ndim,
    eryn_like, 
    priors,
    args=[],
    tempering_kwargs=tempering_kwargs,
    stopping_fn=stop,
    stopping_iterations=10,
    moves=mcmc_moves,
    vectorize=True,
)

In [ ]:
# randomize starting positions throughout prior
coords = start_priors.rvs(size=(ntemps, nwalkers,))
print(coords.shape)

thin_by = 100 
burn = 0
nsteps = int(100000 / thin_by) # should be more than enough 

ensemble.stopping_fn = None

out = ensemble.run_mcmc(coords, nsteps, burn=burn, progress=True, thin_by=thin_by)

In [ ]:
# resume run after the run is stopped 
thin_by =100
burn = 0
nsteps = int(100000 / thin_by) # this should be far more than enough 

out = ensemble.run_mcmc(ensemble.get_last_sample(), nsteps, burn=burn, progress=True, thin_by=thin_by)

## Show moves 

In [ ]:
thin = 10
burnin = 0
len_chain = len(ensemble.get_chain(thin=thin, discard=burnin)['model_0'][:, 0, 0, :, 0])

plt.figure()
fig, ax = plt.subplots(ndim, 1)
fig.set_size_inches(10, 3*ndim)
for i in range(ndim):     
    for walk in range(20): # plot 20 walkers 
        ax[i].plot(ensemble.get_chain(thin=thin, discard=burnin)['model_0'][:, 0, walk, 0, i])
        ax[i].hlines(truths[i], 0, len_chain, color='k', linestyle='-.', linewidth=0.8)
    ax[i].set_ylabel(labels[i])
plt.savefig("/home/Triangle-BBH/Examples/MCMC_trajectory_full.jpg")

## Corner plot

In [ ]:
burnin=1500
thin = 10

samp = ensemble.get_chain(discard=burnin, thin=thin)['model_0'][:, 0, :, :, :].reshape(-1, ndim)
print("sample shape:", samp.shape)

plt.figure()
corner(
    samp, bins=50, color=BLUE, 
    labels=labels, label_kwargs={'fontsize': 14}, 
    # range=lims,
    truths=truths, truth_color=RED, 
    quantiles=[0.16, 0.5, 0.84],
    show_titles=True, title_kwargs={'fontsize':14},
    levels = (1. - np.exp(-1.**2/2), 1. - np.exp(-2.**2/2), 1. - np.exp(-3.**2/2)),
    smooth=0.9, # default for bilby: smooth = 0.9, bins = 50 
    plot_density=True, # whether to show the density of samples with colors 
    plot_datapoints=False, # whether to plot individual data points 
    fill_contours=True, # whether to fill the corners 
    );
plt.savefig("/home/Triangle-BBH/Examples/MCMC_corner_full.jpg")
np.save("chain_full.npy", samp)